In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

from pprint import pprint as pp

In [ ]:
#Aug 25 - Non-clipping
#Sept 14 - Clipping 
#ALl of August

df1 = pd.read_csv("C:\\Users\\dpinales\\Desktop\\Energy_Opt\\data\\Eglin_Analysis\\Eglin - 9-14 CBX1.csv")
df2 = pd.read_csv("C:\\Users\\dpinales\\Desktop\\Energy_Opt\\data\\Eglin_Analysis\\Eglin - 9-14 CBX2.csv")
df3 = pd.read_csv("C:\\Users\\dpinales\\Desktop\\Energy_Opt\\data\\Eglin_Analysis\\Eglin - 9-14_MTR_POA.csv")

df_index = df1['Date (-05:00)']


In [ ]:
all_df = pd.concat([df1.reset_index(drop=True),
                    df2.reset_index(drop=True), 
                    df3.reset_index(drop=True)], axis=1,ignore_index=True)

df_columns = [list(df1.columns),
              list(df2.columns),
              list(df3.columns),]

flatten = lambda nested_list: [item for sublist in nested_list for item in sublist]
all_df.columns = flatten(df_columns)

for i in all_df.columns:
    if "Date (-05:00)" in i:
        new_df = all_df.drop([i], axis=1)


new_df = new_df.set_index([df_index])
new_df.index = pd.to_datetime(new_df.index)
new_df = new_df[~new_df.index.duplicated(keep='first')]
new_df

In [ ]:
''' 
Filters:
        1. No Negative values for CBX
        2. Meter filtered for 10% of MAX for MIN and 90% of MAX for Clipping
        3. Irradiance above 200
        4. Start at 11 am, Peak hours/No Shading
Added Parameters: 
        5. DC Expected = daily_poa*string_count*IMP
        6. Heat Map: Summation(DC_Current)/DC_Expected
'''


for i in new_df.columns:
    if "Recombiner" in i:
        filtered_df1 = new_df[new_df.loc[:,i] > 1]
        
meter_min = 30000000 * 0.10
meter_max = 30000000 * 0.98


filtered_df3 = filtered_df1[(filtered_df1.loc[:, "Meter AC Power (W)"]>meter_min)&(filtered_df1.loc[:, "Meter AC Power (W)"]<meter_max)&(filtered_df1.loc[:,"Sensors - Irradiance reference (W/m²)"]>200)]

filtered_df3 = new_df.between_time("9:00", "17:00")
filtered_df3[filtered_df3 <= 0.00] = 0


In [ ]:

cbx = []
cbx_sum = []


for x in filtered_df3:
    if "Recombiner" in x:
        cbx.append(x)
        cbx_sum.append(filtered_df3[x].sum())
        
cbx_data = pd.DataFrame(cbx_sum, index=[cbx], columns=['Sum_of_CBX'])
cbx_data

In [ ]:
fig_dates = filtered_df3.index
fig_index = cbx_data.index
fig_heatdf = cbx_data.filter(["Estimated_Downstrings", "String_count"])
# fig_heatdf
fig = px.density_heatmap(fig_heatdf, x=fig_heatdf.index.values,y=fig_dates,)
fig.show()


In [ ]:
e_downstrings = sum(down_strings)
total_string = sum(string_count)
ratio_strings = e_downstrings/total_string

print(f"The estimated plants DC outage: {ratio_strings}")

In [ ]:
inverters = pd.read_csv("data\\Eglin - Nonclipping_8-25 INV.csv")
inverters = inverters.set_index([df_index])
inverters.index = pd.to_datetime(inverters.index)
inverters = inverters[~inverters.index.duplicated(keep='first')]
inverters = inverters.drop("Date (-05:00)", axis=1)
inverters = inverters.between_time("9:00", "17:00")

inv_sum = []
inv = []


for x in inverters.columns:
    if "INV" in x:
        inv.append(x)
        inv_sum.append(inverters[x].sum())
inv_df = pd.DataFrame(inv_sum, index=[inv], columns=['Sum_of_inv'])
inv_df

In [ ]:
'''Summation of CMB/INV'''


sum_of_cbx = cbx_data['Sum_of_CBX']
sum_of_inv = inv_df['Sum_of_inv']

def cmb_summation(a,b):
    total = (a/b)
    return total


cmb1 = []
blk1_sum = []
_summation1 = []

inv1 = []
inv1_blk = []

for inv_range in range(100, 108, 1):
    inv1.append(sum_of_inv.filter(like=str(inv_range), axis=0))

for i in range(1, len(inv1)):
    _inv1 = (sum(inv1[i]))
    inv1_blk.append(_inv1)    

for cmb_range in range(100, 108, 1):
    cmb1.append(sum_of_cbx.filter(like=str(cmb_range), axis=0))

for i in range(1,len(cmb1)):
    _cb1 = (sum(cmb1[i]))
    blk1_sum.append(_cb1)

for (v, n) in zip(blk1_sum, inv1_blk):
    _summation1.append(cmb_summation(v, n))
    

In [ ]:
inv2_dc = {
    "INV1",
    "INV2",
    "INV3",
    "INV4",
    "INV5",
    "INV6",
    "INV7",
}


cmb2 = []
blk2_sum = []

inv2 = []
inv2_blk = []

_summation2 = []

for inv_range in range(200, 208, 1):
    inv2.append(sum_of_inv.filter(like=str(inv_range), axis=0))

for i in range(1, len(inv2)):
    _inv2 = (sum(inv2[i]))
    inv2_blk.append(_inv2)    

for cmb_range in range(200, 208, 1):
    cmb2.append(sum_of_cbx.filter(like=str(cmb_range), axis=0))

for i in range(1,len(cmb2)):
    _cb2 = (sum(cmb2[i]))
    blk2_sum.append(_cb2)

for (v, n) in zip(blk2_sum, inv1_blk):
    _summation2.append(cmb_summation(v, n))


In [ ]:
total_summation = list(zip(_summation1, _summation2))

cbx_over_inv = pd.DataFrame(total_summation, index=[n for n in inv2_dc.keys()],columns=['BLK1 CBX100-107/INV DC', "BLK2 CBX200-207/INV DC"])
cbx_over_inv

In [ ]:
cbx_data.to_csv("Gulf I - 9-14.csv")